In [27]:
%cd /Users/madhav/module-1-biomarker-data-table_seven

/Users/madhav/module-1-biomarker-data-table_seven


In [80]:
#loading in training data
import pandas as pd
training_data = pd.read_csv('data/biomarker-train.csv')

In [81]:
#loading in testing data
test_data = pd.read_csv('data/biomarker-test.csv')
test_data = test_data.drop(columns=['Target Full Name', 'ADOS        Total Score'])

In [82]:
test_data = test_data.dropna()

In [84]:
training_data.head()

,Group,Target Full Name,E3 ubiquitin-protein ligase CHIP,CCAAT/enhancer-binding protein beta,Gamma-enolase,E3 SUMO-protein ligase PIAS4,Interleukin-10 receptor subunit alpha,Signal transducer and activator of transcription 3,Interferon regulatory factor 1,Transcription factor AP-1,...,Transgelin-2,"ATP synthase subunit O, mitochondrial",Pro-opiomelanocortin,Quinone oxidoreductase-like protein 1,Pigment epithelium-derived factor,Cathepsin F,Formimidoyltransferase-cyclodeaminase,Ubiquitin carboxyl-terminal hydrolase 25,Plexin-B2,ADOS Total Score
0,ASD,NaN,458.9,1367.2,780.8,905.7,2244.0,450.2,1469.7,3401.5,...,3365.9,1424.5,601.4,2608.2,24093.8,2600.0,1780.3,1663.8,2629.7,22
1,ASD,NaN,618.6,1489.3,732.7,1229.6,1647.0,467.0,1041.0,3114.2,...,3016.2,2156.9,895.7,2313.6,24904.5,2048.7,9942.4,1462.6,2024.1,8
2,ASD,NaN,916.8,1716.5,877.2,978.6,1970.6,501.0,1434.8,3013.8,...,4124.4,1488.3,688.7,2466.8,24585.4,3737.1,2421.2,1670.7,2525.2,22
3,ASD,NaN,458.3,1021.1,1142.3,740.1,1669.8,372.4,1022.4,2623.5,...,2962.8,1731.0,525.3,1579.9,30119.9,2453.0,307.7,1444.9,1648.8,7
4,ASD,NaN,395.6,1356.4,638.7,1044.6,1821.8,326.2,1085.7,2696.7,...,3327.2,1708.2,585.0,2193.7,23211.6,2667.9,1032.4,1478.1,1853.7,21


In [85]:
training_data = training_data.drop(columns=['Target Full Name', 'ADOS        Total Score'])

In [86]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

predictors = training_data.drop(columns=['Group'])
response = training_data['Group']

# Fit Random Forest
rf = RandomForestClassifier(
    n_estimators=1000,
    random_state=101422,
    oob_score=False, 
)

rf.fit(X=predictors, y=response)

#confusion matrix
y_pred = rf.predict(predictors)
conf_mat = confusion_matrix(response, y_pred)
print(conf_mat)

# Compute feature importance
importances = pd.DataFrame({
    'protein': predictors.columns,
    'MeanDecreaseGini': rf.feature_importances_
})

# Get top 10 proteins
proteins_s2 = (
    importances
    .nlargest(10, 'MeanDecreaseGini')
    ['protein']
    .tolist()
)

print(proteins_s2)

[[61  0]
 [ 0 62]]
['Dermatopontin', 'Immunoglobulin D', 'C-X-C motif chemokine 16', 'Pleiotrophin', 'Mitogen-activated protein kinase 14', 'CD166 antigen', 'Hyaluronan and proteoglycan link protein 1', 'Coagulation factor IX', 'Neurogenic locus notch homolog protein 1', 'Dickkopf-related protein 3']


In [88]:
#performance with a simple subset (top 8 proteins)
simple_proteins = list(['Dermatopontin', 'Immunoglobulin D', 'C-X-C motif chemokine 16', 'Pleiotrophin', 'Mitogen-activated protein kinase 14', 'CD166 antigen', 'Hyaluronan and proteoglycan link protein 1', 'Coagulation factor IX'])

predictors_simple = training_data[simple_proteins]
rf_simple = RandomForestClassifier(
    n_estimators=1000,
    random_state=101422,
    oob_score=False, 
)
rf_simple.fit(X=predictors_simple, y=response)

X_test_simple = test_data.drop(columns=['Group'])
X_test_simple = X_test_simple[simple_proteins]

y_test_simple = test_data['Group']

y_pred_simple = rf_simple.predict(X_test_simple)

conf_mat_simple = confusion_matrix(y_test_simple, y_pred_simple)

accuracy_simple = np.trace(conf_mat_simple) / np.sum(conf_mat_simple)
print("Accuracy:", accuracy_simple)

Accuracy: 0.8064516129032258


In [91]:
conf_mat_simple

array([[13,  2],
       [ 4, 12]])

In [92]:
conf_mat_s2

array([[12,  3],
       [ 3, 13]])